In [ ]:
#instalando pymongo
!pip install pymongo

In [1]:
#Biblioteca usada para requisitar uma pagina web
import urllib.request
import pandas as pd
from pymongo import MongoClient

In [2]:
# Definimos a url
# Verifique as permissões em https://www.python.org/robots.txt
with urllib.request.urlopen('https://www.worldfootball.net/attendance/esp-primera-division-2017-2018/1/') as url:
    page = url.read()

In [3]:
from bs4 import BeautifulSoup

In [4]:
# Analise o html na variável 'page' e armazene-o no formato Beautiful Soup
soup = BeautifulSoup(page, "html.parser")

In [5]:
soup.title

<title>Primera División 2017/2018 » Attendance » Home matches</title>

In [ ]:
soup.find_all('table')

In [20]:
rodada = soup.find_all('option',{'class':"wahl"})[1]
rodada = rodada.get_text(strip=True).split('.')[0]

'1'

In [11]:
#selecionando tabela com a classe
Classif = soup.find_all('table',{'class':"standard_tabelle"})[0]

In [12]:
Cabecalho = Classif.find_all('tr')

In [35]:
header = Classif.find('tr')

In [43]:
header.get_text(strip=True)

'\n#\nTeam\nsum\nMatches\naverage\n'

In [37]:
linha = Classif.find_all('tr')[1:]

In [46]:
header_scorer = []
for nome in header:
    header_scorer.append([cab.get_text(strip=True) for cab in nome.find_all('td')]) 
    #print (cab.)

AttributeError: 'NavigableString' object has no attribute 'find_all'

In [49]:
header = ['Rank','campo_vazio','Time','Total','Jogos','Media']

In [44]:
jogos=[]
#pos = [0,1,2,4,5]
for jogo in linha:
    jogos.append([partida.get_text(strip=True) for partida in jogo.find_all('td')])
    
print (jogos)

[['1', '', 'FC Barcelona', '1.248.898', '19', '65.731'], ['2', '', 'Real Madrid', '1.235.513', '19', '65.027'], ['3', '', 'Atlético Madrid', '1.054.252', '19', '55.487'], ['4', '', 'Real Betis', '881.470', '19', '46.393'], ['5', '', 'Valencia CF', '735.273', '19', '38.699'], ['6', '', 'Athletic Bilbao', '710.175', '19', '37.378'], ['7', '', 'Sevilla FC', '628.314', '19', '33.069'], ['8', '', 'Deportivo La Coruña', '392.050', '19', '20.634'], ['9', '', 'Málaga CF', '387.618', '19', '20.401'], ['10', '', 'Real Sociedad', '374.514', '19', '19.711'], ['11', '', 'Levante UD', '336.029', '19', '17.686'], ['12', '', 'Espanyol Barcelona', '334.286', '19', '17.594'], ['13', '', 'Villarreal CF', '316.602', '19', '16.663'], ['14', '', 'Celta Vigo', '308.822', '19', '16.254'], ['15', '', 'UD Las Palmas', '305.931', '19', '16.102'], ['16', '', 'CD Alavés', '286.127', '19', '15.059'], ['17', '', 'Girona FC', '193.835', '19', '10.202'], ['18', '', 'Getafe CF', '193.736', '19', '10.197'], ['19', '', '

In [50]:
Partidas = pd.DataFrame.from_records(jogos,columns = header)
Partidas.dropna(inplace=True)
Partidas.head()

,Rank,campo_vazio,Time,Total,Jogos,Media
0,1,,FC Barcelona,1.248.898,19,65.731
1,2,,Real Madrid,1.235.513,19,65.027
2,3,,Atlético Madrid,1.054.252,19,55.487
3,4,,Real Betis,881.470,19,46.393
4,5,,Valencia CF,735.273,19,38.699


In [31]:
Partidas.head()

,Data,Hora,Home,campo1,Away,Placar,campo2,campo3
0,11/08/2017,19:45,Arsenal FC,-,Leicester City,4:3 (2:2),,
1,12/08/2017,12:30,Watford FC,-,Liverpool FC,3:3 (2:1),,
2,,15:00,Chelsea FC,-,Burnley FC,2:3 (0:3),,
3,,15:00,Crystal Palace,-,Huddersfield Town,0:3 (0:2),,
4,,15:00,Everton FC,-,Stoke City,1:0 (1:0),,


In [33]:
placar = [placar[0] for placar in Partidas['Placar'].str.split('(').tolist()]
Partidas['Placar'] = placar
Partidas.head()    

,Data,Hora,Home,campo1,Away,Placar,campo2,campo3
0,11/08/2017,19:45,Arsenal FC,-,Leicester City,4:3,,
1,12/08/2017,12:30,Watford FC,-,Liverpool FC,3:3,,
2,,15:00,Chelsea FC,-,Burnley FC,2:3,,
3,,15:00,Crystal Palace,-,Huddersfield Town,0:3,,
4,,15:00,Everton FC,-,Stoke City,1:0,,


In [30]:
import re

In [66]:
Partidas['points'] = Partidas['points'].str.split('+')


In [44]:
ab = pd.Series(['a1', 'b2', 'c3'])
ab.str.extract('([ab])(\d)', expand=False)

,0,1
0,a,1
1,b,2
2,NaN,NaN


In [71]:
gols = [valor[0] for valor in Partidas['points']]
ass = [valor[1] for valor in Partidas['points']]
Partidas['gols'] = gols
Partidas['ass'] = ass


In [76]:
Partidas.head()

,#,Player,,Country,Team,points,gols,ass
1,1.,Eden Hazard,,Belgium,Chelsea FC,"[10(7, 3)]",10(7,3)
2,2.,Kun Agüero,,Argentina,Manchester City,"[7(5, 2)]",7(5,2)
3,,Callum Wilson,,England,AFC Bournemouth,"[7(3, 4)]",7(3,4)
4,4.,Harry Kane,,England,Tottenham Hotspur,"[6(5, 1)]",6(5,1)
5,,Raheem Sterling,,England,Manchester City,"[6(4, 2)]",6(4,2)


In [ ]:
Partidas['ass'] = Partidas['ass'].str.split(')',expand=True)[0]

In [ ]:
Partidas['gols'] = Partidas['gols'].str.split('(',expand=True)[1]


In [22]:
#Formata o placar
placar = [placar[0] for placar in Partidas['Placar'].str.split('(').tolist()]
Partidas['Placar'] = placar


,Data,Hora,Home,Away,Placar,Placar2
0,10/08/2018,20:00,Manchester United,Leicester City,2:1 (1:0),2:1
1,11/08/2018,12:30,Newcastle United,Tottenham Hotspur,1:2 (1:2),1:2
2,,15:00,AFC Bournemouth,Cardiff City,2:0 (1:0),2:0
3,,15:00,Fulham FC,Crystal Palace,0:2 (0:1),0:2
4,,15:00,Huddersfield Town,Chelsea FC,0:3 (0:2),0:3
5,,15:00,Watford FC,Brighton & Hove Albion,2:0 (1:0),2:0
6,,17:30,Wolverhampton Wanderers,Everton FC,2:2 (1:1),2:2
7,12/08/2018,13:30,Southampton FC,Burnley FC,0:0 (0:0),0:0
8,,13:30,Liverpool FC,West Ham United,4:0 (2:0),4:0
9,,16:00,Arsenal FC,Manchester City,0:2 (0:1),0:2


In [ ]:
#Selecionando cabecalho
head = []
for rows in Cabecalho:
    head.append(rows.get_text(strip=True))
    #print(type(rows))
    

In [ ]:
Classif.find_all('tr')

In [ ]:
#Selecionando dados para criacao do dataset
dados = []
for linha in Classif.find_all('tr')[1:]:
    dados.append([campo.get_text(strip=True) for campo in linha.find_all('td') ])
   

In [ ]:
dados_ok=[]
for d in dados:
    d.pop(1)
    dados_ok.append(d)

In [ ]:
Tabela_PremierLeague = pd.DataFrame.from_records(dados_ok,columns = head)

In [ ]:
Tabela_PremierLeague.columns

In [ ]:
Tabela_PremierLeague.rename(columns={'#':'rank','M.':'M','Dif.':'Dif','Pt.':'Pt'},inplace=True)

In [ ]:
#Cria coluna rodada
Tabela_PremierLeague['Rodada'] = rodada

In [50]:
conn = MongoClient('localhost',27017)

In [22]:
type(conn)

pymongo.mongo_client.MongoClient

In [51]:
#Criacao do Banco
db = conn.RankGeral

In [52]:
#Cricao da colecao
Premier_collection = db.PremierLeague_Table

In [ ]:
type(collection)

In [ ]:
PremierLeagueDict = Tabela_PremierLeague.to_dict('records')

In [ ]:
type(PremierLeagueDict)

In [ ]:
#inserindo dados na colecao
resultado = Premier_collection.insert_many(PremierLeagueDict)

In [ ]:
resultado.inserted_ids

In [53]:
#lista as colections do banco
db.list_collection_names()

['PremierLeague_Table']

In [25]:
#Exclui a collection
db.Tabelas.drop()

In [57]:
#lista primeiro registro
Premier_collection.find_one()

{'D': '0',
 'Dif': '7',
 'L': '0',
 'M': '2',
 'Pt': '6',
 'Rodada': '2',
 'Team': 'Manchester City',
 'W': '2',
 '_id': ObjectId('5bb952de81a0d535e12db2be'),
 'goals': '8:1',
 'rank': '1'}

In [ ]:
#lista todos os registros
for valor in Premier_collection.find():
    print(valor)

In [63]:
#Lista documentos
Premier_collection.estimated_document_count()

0

In [61]:
Premier_collection.find_one()

In [62]:
#remove todos os documentos
Premier_collection.delete_many({})

In [63]:
#Lista documentos
for rodadas in Premier_collection.find({'M':{'$gt':'1'}}):
    print(rodadas)

{'_id': ObjectId('5bb01ebe81a0d515c1895e18'), 'rank': '1', 'Team': 'Manchester City', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '8:1', 'Dif': '7', 'Pt': '6'}
{'_id': ObjectId('5bb01ebe81a0d515c1895e19'), 'rank': '2', 'Team': 'Liverpool FC', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '6:0', 'Dif': '6', 'Pt': '6'}
{'_id': ObjectId('5bb01ebe81a0d515c1895e1a'), 'rank': '3', 'Team': 'Chelsea FC', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '6:2', 'Dif': '4', 'Pt': '6'}
{'_id': ObjectId('5bb01ebe81a0d515c1895e1b'), 'rank': '4', 'Team': 'Watford FC', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '5:1', 'Dif': '4', 'Pt': '6'}
{'_id': ObjectId('5bb01ebe81a0d515c1895e1c'), 'rank': '5', 'Team': 'Tottenham Hotspur', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '5:2', 'Dif': '3', 'Pt': '6'}
{'_id': ObjectId('5bb01ebe81a0d515c1895e1d'), 'rank': '6', 'Team': 'AFC Bournemouth', 'M': '2', 'W': '2', 'D': '0', 'L': '0', 'goals': '4:1', 'Dif': '3', 'Pt': '6'}
{'_id': ObjectId('5bb